In [129]:
import os, json,shutil
import requests, jinja2
import random
from bs4 import BeautifulSoup
import time
from time import gmtime, strftime
from datetime import datetime

#globals (via a config later on)
git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'
sdk_url = 'https://api.github.com/repos/Azure/azure-sdk-for-python/'
assumed_current_date = '2017-04-01' #all packages without build.json are assued to be current as of  04-01
sdk_raw_url = 'https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/' 

with open('config/api2sdk2nuget.json', 'r') as f:
    map_object = json.load(f)
    
sdk_map = map_object


#get existing projects from swagger_to_sdk config file (this is the main source of truth.)

swagger_to_sdk_config_file_name = 'swagger_to_sdk_config.json'

swagger_to_sdk = request_helper(sdk_raw_url + swagger_to_sdk_config_file_name )


#azure_projects = [get_azure_name_space_data(swagger_to_sdk['projects'][p]['swagger'])[0] for p in swagger_to_sdk['projects']]
azure_projects = [parse_swagger_to_sdk_config(swagger_to_sdk['projects'][p]) for p in swagger_to_sdk['projects']]
azure_projects_no_duplicate = list(set(azure_projects))



## FIXES

In [139]:
x = request_helper(git_url + 'contents/' + 'random' )

In [13]:
raw_url, git_url, swagger_to_sdk_config_file_name, sdk_url, assumed_current_date = None, None, None, None, None
#pungi 
if not raw_url:
    raw_url = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/master/' 
if not sdk_raw_url:
    sdk_raw_url = 'https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/' 


if not swagger_to_sdk_config_file_name:
    swagger_to_sdk_config_file_name = 'swagger_to_sdk_config.json'

if not assumed_current_date:
    assumed_current_date = '2017-04-01' #all packages without build.json are assued to be current as of  04-01

if not git_url:
    git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'


if not sdk_url:
    sdk_url = 'https://api.github.com/repos/Azure/azure-sdk-for-python/'


existing_changes ={}
existing_changes['errors'] = {}

#get main swagger_to_sdk_config file 

swagger_to_sdk = request_helper(sdk_raw_url + swagger_to_sdk_config_file_name )

for proj in swagger_to_sdk['projects']:

    #if proj.startswith('rdbms.mysql'):

    print ('printing_prject_name :' + proj )

    #(u'arm-commerce', u'2015-06-01-preview', u'commerce.json'), (u'arm-network', 'Composite', u'compositeNetworkClient_2015_06_15.json')

    azure_api_name, c_composite, c_swagger, sdk, namespace = parse_swagger_to_sdk_config(swagger_to_sdk['projects'][proj])

    current_swagger_path = swagger_to_sdk['projects'][proj]['swagger']

    sdk_recent_build = get_python_sdk_build_info(sdk)

    #print(sdk_recent_build)

    if not sdk_recent_build:
        c_recent_date = assumed_current_date #assume it's current as of April 15th. 
        c_version = swagger_to_sdk['projects'][proj]['autorest_options'].get("PackageVersion", "0.00") #assume current version == package version

    else:
        c_recent_date = sdk_recent_build['date']
        c_version = sdk_recent_build['version']

    #print c_recent_date, c_version

    if not existing_changes.get(sdk):
        i = sdk 
        existing_changes[i] ={}
        existing_changes[i]['meta'] = {'azure_api_name' : azure_api_name, 'composite_or_recent_folder' : c_composite, 
                           'current_swagger': current_swagger_path , 'recent_build_date': c_recent_date, 
                           'current_version':c_version}

    else:
        #there is already an object with the same sdk name. ..this is multiple. 
        i= namespace
        existing_changes[i] = {}
        existing_changes[i]['meta'] = {'azure_api_name' : azure_api_name, 'composite_or_recent_folder' : c_composite, 
               'current_swagger': c_swagger , 'recent_build_date': c_recent_date, 
               'current_version':c_version}
        existing_changes[i]['multiple'] = 'yes'
        existing_changes[i]['parent_sdk'] = sdk

    #GET THE info from azure api spec for this project. 

    params = get_key_folder_params_v3(git_url,azure_api_name)

    if not params:
        existing_changes[i]['errors'] = {'Project' : 'Project ' + azure_api_name +' not found. Probably private' }
        existing_changes['errors'][i] = 'Project ' + azure_api_name +' not found. Probably private'

    else:
        is_composite, folders, swagger = get_key_folder_params_v3(git_url,azure_api_name)

        if c_composite =='Composite':
            current_composite = 'Yes'
            c_recent_folder = 'No'

        else:
            c_recent_folder = c_composite
            current_composite = 'No'

        #print(azure_api_name, c_composite, c_swagger)
        #print(is_composite, folders, swagger )


        #check changes in composite status. 
        if is_composite != current_composite:
            #project composite status has changed. 
            changes = get_swagger_updates(swagger, git_url=git_url) 
            changes['use_swagger'] = swagger

            if current_composite =="Yes":
                #is_composite is NO

                print(azure_api_name + 'has changed to Composite to --> non composite')  

                #get the details for this swagger file and update the details. 

                changes['change_type'] = "CompositeStatus"
                changes['change_status'] = "Moved to a non - composite swagger"
                existing_changes[i]['changes'] = changes


            else:
                #moved to a composite swagger. get all lower level swagger details if possible. 

                print(azure_api_name + 'Moved to a composite swagger')
                changes['change_type'] = "CompositeStatus"
                changes['change_status'] = "Moved to a composite swagger"

                existing_changes[i]['changes']= changes

        #check if folder is the same 
        else:     
            if is_composite =='No': 
                #there must be folders. 
                if len(folders) > 0: 
                    latest_folder = folders[-1]
                    if c_recent_folder != latest_folder:

                        print(azure_api_name + ' has a new folder : ' + latest_folder)

                        changes = get_swagger_updates(swagger, git_url=git_url)
                        changes['use_swagger'] = swagger
                        changes['change_type'] = "Folder"
                        changes['new_folder'] = latest_folder
                        existing_changes[i]['changes'] =changes

                    else:
                        #most proable scenario : check for swagger update. 

                        if current_swagger_path != swagger:
                            print (azure_api_name + '   swagger not found')
                            #print(c_swagger, swagger)

                        else:
                            #get the current sha of from the recent date. 

                            changes = get_swagger_updates_v2(current_swagger_path, git_url=git_url, current_date=c_recent_date)

                            if changes['swagger_behind'] >0:
                                changes['change_type'] = "SwaggerUpdate"
                                existing_changes[i]['changes'] = changes


            else:
                #TO DO TO DO TO DO existing composite project that may have been updated. check if swagger has changed. 
                #i.e, a composite project. 
                #current_sha = base_data['current_version']
                print ('COMPOSITE_SWAGGER' + current_swagger_path)
                if not c_swagger or not current_swagger_path:
                        print (azure_api_name + 'swagger not found')
                else:
                    # check for the update in main composite file first 
                    changes_composite = get_swagger_updates_v2(current_swagger_path, git_url=git_url, current_date=c_recent_date)

                    if changes_composite['swagger_behind'] >0:
                        changes['change_type'] = "SwaggerUpdate"
                        existing_changes[i]['changes'] = changes

                    # check for the update in main composite file first                 
                    #get the full file and individual paths from the composite file. 
                    #raw_url = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/master/' 

                    cfile= request_helper(raw_url + current_swagger_path)

                    if not cfile:
                        #current swagger file not found. 

                        existing_changes['errors'][i] = 'swagger_file : ' + current_swagger_path +' not found'

                        if not existing_changes[i].get('errors'): 
                            existing_changes[i]['errors'] = {}
                            existing_changes[i]['errors']['swagger_file'] = 'swagger_file : ' + current_swagger_path +' not found'

                        else:
                            existing_changes[i]['errors']['swagger_file'] = 'swagger_file : ' + current_swagger_path +' not found'


                    if cfile and cfile.get('documents'):
                        for f in cfile.get('documents'):
                            #get file history 
                            ind_swagger = azure_api_name + f[1:]
                            print ('IND_SWAGGER' + ind_swagger)
                            #file_data = get_swagger_updates(ind_swagger, git_url, current_sha=None)
                            changes_ind_file = get_swagger_updates_v2(ind_swagger, git_url=git_url, current_date=c_recent_date)

                            #catch any errors. 

                            if not changes_ind_file:
                                existing_changes['errors'][i] = 'swagger_file : ' + ind_swagger +' not found'

                            if changes_ind_file and changes_ind_file['swagger_behind'] >0:

                                if existing_changes[i].get('changes'):

                                    if not existing_changes[i]['changes'].get('ind_changes'):
                                        existing_changes[i]['changes']['ind_changes'] = {}
                                        existing_changes[i]['changes']['ind_changes'][ind_swagger] = changes_ind_file
                                    else:
                                         existing_changes[i]['changes']['ind_changes'][ind_swagger] = changes_ind_file


                            #d[ind_swagger] = {'sha' : file_data['commit_sha'], 'dates': file_data['file_dates']}

        #print('CHANGES')

        if not sdk_map.get(azure_api_name):
            existing_changes[i]['nuget_info'] = {}
            print('No Nuget URL Map for azure api: ' + azure_api_name)

        else:
            nuget_package = sdk_map[azure_api_name].get('nuget_package')
            if nuget_package:
                existing_changes[i]['nuget_info'] = get_recent_from_nuget_v2(nuget_package) or 'Nuget info not found.'
            else:
                existing_changes[i]['nuget_info'] = {}
                print('No Nuget URL Map for azure api: ' + azure_api_name)

    #update multiple projects for easy rendering later on. 

    update_multiple_projects_4_web(existing_changes)

print existing_changes


printing_prject_name :compute.compute.2015-06-15
arm-computeMoved to a composite swagger
printing_prject_name :trafficmanager
printing_prject_name :graphrbac
COMPOSITE_SWAGGERarm-graphrbac/compositeGraphRbacManagementClient.json
IND_SWAGGERarm-graphrbac/1.6/swagger/graphrbac.json
printing_prject_name :cdn
printing_prject_name :iothub
arm-iothub has a new folder : 2017-01-19
printing_prject_name :batch
No Nuget URL Map for azure api: batch
printing_prject_name :resources.links.2016-09-01
printing_prject_name :servermanager
printing_prject_name :network.2015-06-15
COMPOSITE_SWAGGERarm-network/compositeNetworkClient_2015_06_15.json
printing_prject_name :eventhub
printing_prject_name :rdbms.mysql
printing_prject_name :network.2016-09-01
COMPOSITE_SWAGGERarm-network/compositeNetworkClient_2016_09_01.json
printing_prject_name :datalake.analytics.catalog
printing_prject_name :documentdb
No Nuget URL Map for azure api: arm-documentdb
printing_prject_name :storage.2016-12-01
printing_prject_nam

In [139]:
#sdk_recent_build = get_python_sdk_build_info(sdk)
x = swagger_to_sdk['projects']['billing']
print(parse_swagger_to_sdk_config(x))
print (get_python_sdk_build_info('azure-mgmt-billing'))
#azure_api_name, c_composite, c_swagger, sdk, namespace = parse_swagger_to_sdk_config(swagger_to_sdk['projects'][proj])

(u'arm-billing', u'2017-04-24-preview', u'billing.json', u'azure-mgmt-billing', u'azure.mgmt.billing')
None


In [141]:
for p in swagger_to_sdk['projects']:
    if 'network' in p:
        print p
        print(parse_swagger_to_sdk_config(swagger_to_sdk['projects'][p]))

network.2015-06-15
(u'arm-network', 'Composite', u'compositeNetworkClient_2015_06_15.json', u'azure-mgmt-network', u'azure.mgmt.network.v2015_06_15')
network.2016-09-01
(u'arm-network', 'Composite', u'compositeNetworkClient_2016_09_01.json', u'azure-mgmt-network', u'azure.mgmt.network.v2016_09_01')
network.2017-03-01
(u'arm-network', 'Composite', u'compositeNetworkClient.json', u'azure-mgmt-network', u'azure.mgmt.network.v2017_03_01')
network.2016-12-01
(u'arm-network', 'Composite', u'compositeNetworkClient_2016_12_01.json', u'azure-mgmt-network', u'azure.mgmt.network.v2016_12_01')


In [30]:
sha2pr = {}
missing_pr_shas =[]
missing_pr_projects =[]

for p in existing_changes:
    if existing_changes[p].get('changes'):
        if existing_changes[p]['changes'].get('commit_sha'):   
            if not existing_changes[p]['changes'].get('pr_num') or existing_changes[p]['changes']['pr_num'] == 'not found':
                r_commit = existing_changes[p]['changes']['commit_sha'][0]
                #check if r_commit is already is in the file.        
                if sha2pr.get(r_commit):
                    existing_changes[proj]['changes']['pr_num'] = sha2pr[r_commit]

                else:
                    missing_pr_shas.append(r_commit)
                    missing_pr_projects.append(p)

print missing_pr_shas
print len(missing_pr_shas)

[u'4779d58229fbe5cb94e8c6c40204af0515c07677', u'156cd442edf6badb940149878cecfba2d3198776', u'cf35632382557982d526792bbc78d3e50888ff7e', u'12b641f181711ec1e9a50f1dcfc7861721aa3242', u'68c601d4a1fcc45f81f6bb5d2160ade962012ddf', u'20b02b6d2f6cdc6bd7f627bec57fc8b0bde12e26', u'26837ac3ab19dcd537ceb5805a4cbbfce205cc1b', u'ba50eafc653f8d6340bb261fd0da81dc1929a3be', u'ab6034c2ed4ae7347a5817242487706e5a49b73c']
9


In [14]:
#write above for testig
with open('write-out-changes.json', 'w') as f:
    json.dump(existing_changes, f)

In [12]:
#existing_projects = update_remaining_PR(existing_changes)
for k,v in existing_changes.items():
    if v.get('changes'):
        print k

azure.mgmt.storage.v2015_06_15
azure.mgmt.resource.policy.v2015_10_01_preview
azure-mgmt-servicebus
azure-mgmt-billing
azure-mgmt-datalake-store
azure-mgmt-storage
azure.mgmt.resource.resources.v2016_02_01
azure.mgmt.datalake.analytics.account
azure-mgmt-documentdb
azure-mgmt-iothub
azure-mgmt-notificationhubs
azure-mgmt-cognitiveservices
azure-mgmt-eventhub
azure.mgmt.resource.policy.v2016_04_01
azure.mgmt.resource.locks.v2015_01_01
azure-mgmt-resource
azure-mgmt-consumption
azure-mgmt-sql
azure-mgmt-compute
azure-keyvault
azure-mgmt-containerregistry
azure.mgmt.compute.compute.v2016_03_30
azure-mgmt-monitor
azure.mgmt.resource.resources.v2016_09_01
azure-mgmt-keyvault
azure-mgmt-datalake-analytics
azure-servicefabric
azure.mgmt.compute.containerservice.v2017_01_31
unreleased


In [46]:
#get new projects 

with open('changes/archived_2017_05_08.json', 'r') as f:
    old_data = json.load(f)
    
new_projects = old_data['new_projects']
new_projects

{u'arm-advisor': {u'commit_dates': [u'2017-04-25T23:06:28Z',
   u'2017-04-25T17:50:09Z',
   u'2017-04-25T16:45:51Z',
   u'2017-04-24T18:31:14Z',
   u'2017-04-24T15:50:00Z',
   u'2017-04-24T15:39:19Z',
   u'2017-04-21T22:32:52Z',
   u'2017-04-21T19:32:32Z',
   u'2017-04-21T18:39:58Z',
   u'2017-04-21T18:39:16Z',
   u'2017-04-21T00:40:55Z',
   u'2017-04-20T20:06:15Z',
   u'2017-04-20T17:38:52Z',
   u'2017-04-20T16:42:00Z',
   u'2017-04-20T15:21:36Z',
   u'2017-04-20T01:14:25Z',
   u'2017-04-19T21:54:02Z',
   u'2017-04-19T20:30:59Z',
   u'2017-04-19T20:23:35Z',
   u'2017-04-19T10:26:29Z',
   u'2017-04-18T23:51:35Z',
   u'2017-04-18T21:29:53Z',
   u'2017-04-18T17:53:11Z',
   u'2017-04-18T00:42:07Z',
   u'2017-04-18T00:41:32Z',
   u'2017-04-17T21:44:39Z',
   u'2017-04-17T18:03:58Z',
   u'2017-04-14T21:41:54Z',
   u'2017-04-14T21:41:20Z',
   u'2017-04-14T21:39:51Z'],
  u'commits': [u'17b699a8e88234c90b293f1c3a1b6a33d5cfc6c5',
   u'1808e99944e2c6eabcf7f40141b0ab4e969bd4b5',
   u'2d39ac66cec2d

In [130]:
## CREATE REPORT actual function to run everytime. 
#pungi
with open('config/api2sdk2nuget.json', 'r') as f:
    map_object = json.load(f)

with open('config/fudge.json', 'r') as f:
    build_file = json.load(f)
    

#existing_projects = existing_changes
print ('@@@@UPDATING FINDING CHANGES TO EXISTING PROJECTS .....')
existing_projects = get_existing_changes_v3(sdk_map, git_url=git_url, assumed_current_date=assumed_current_date)


import datetime
import time
report_time =  datetime.datetime.now().strftime("%B %d, %Y")

from jinja2 import Environment, FileSystemLoader
env = Environment(loader=FileSystemLoader('templates'))
template = env.get_template('jinja-template_v7.html')

seq = [k for k in sorted(existing_projects.iterkeys())]

number_changes = sum([1 if existing_projects[c].get('changes') and not existing_projects[c].get('multiple')  else 0 for c in existing_projects ])
errors=0

if existing_projects.get('errors'):
    errors = len( [e for e in existing_projects['errors'] ])

print ('@@@@FOUND ....' + str(number_changes) + ' changes to existing projects')

#update the missing PR numbers. 

try:
    with open('config/sha2pr.json', 'r') as f:
        sha2pr = json.load(f)
except:
    sha2pr = {}

print ('@@@@UPDATING PRS .....')
prs = update_remaining_PR_v2(existing_projects, sha2pr=sha2pr) #[[(u'azure-keyvault', u'ab6034c2ed4ae7347a5817242487706e5a49b73c', u'1195')]

print(prs)

for p in prs:
    for p1 in p:
        (proj, sha, pr)=  p1
        sha2pr[sha] = pr 
        if existing_projects[proj].get('changes'):
            existing_projects[proj]['changes']['pr_num'] = pr
            
print('@@@@WRITING UPDATES TO SHA2PR .....')

print sha2pr

with open('config/sha2pr.json', 'w')as f:
    json.dump(sha2pr, f)

#write the update dict object for fture use. 

"""
print('@@@@CREATING OUTPUT FILE .....')


output = template.render(new_projects=new_projects, existing_projects=existing_projects, 
                         recent_sha = '123',recent_date='abc', base=build_file, 
                         sdk_map= map_object, seq = seq, number_changes=number_changes, report_time=report_time, errors=errors)
#print output
#write to file. 
with open('test_report_may11.html', 'w') as f:
    f.write(output)
    
"""

@@@@UPDATING FINDING CHANGES TO EXISTING PROJECTS .....
printing_prject_name :compute.compute.2015-06-15
arm-computeMoved to a composite swagger
printing_prject_name :trafficmanager
arm-trafficmanager has a new folder : 2017-05-01
printing_prject_name :graphrbac
COMPOSITE_SWAGGERarm-graphrbac/compositeGraphRbacManagementClient.json
IND_SWAGGERarm-graphrbac/1.6/swagger/graphrbac.json
printing_prject_name :cdn
printing_prject_name :iothub
arm-iothub has a new folder : 2017-01-19
printing_prject_name :batch
No Nuget URL Map for azure api: batch
printing_prject_name :resources.links.2016-09-01
printing_prject_name :servermanager
printing_prject_name :network.2015-06-15
COMPOSITE_SWAGGERarm-network/compositeNetworkClient_2015_06_15.json
printing_prject_name :eventhub
printing_prject_name :rdbms.mysql
printing_prject_name :network.2016-09-01
COMPOSITE_SWAGGERarm-network/compositeNetworkClient_2016_09_01.json
printing_prject_name :datalake.analytics.catalog
printing_prject_name :documentdb
No

"\nprint('@@@@CREATING OUTPUT FILE .....')\n\n\noutput = template.render(new_projects=new_projects, existing_projects=existing_projects, \n                         recent_sha = '123',recent_date='abc', base=build_file, \n                         sdk_map= map_object, seq = seq, number_changes=number_changes, report_time=report_time, errors=errors)\n#print output\n#write to file. \nwith open('test_report_may11.html', 'w') as f:\n    f.write(output)\n    \n"

In [135]:
#write the existing project. 

print('@@@@CREATING OUTPUT FILE .....')


output = template.render(new_projects=new_projects, existing_projects=existing_projects, 
                         recent_sha = '123',recent_date='abc', base=build_file, 
                         sdk_map= map_object, seq = seq, number_changes=number_changes, report_time=report_time, errors=errors)
#print output
#write to file. 
with open('test_report_may16.html', 'w') as f:
    f.write(output)
    



@@@@CREATING OUTPUT FILE .....


In [114]:
update_remaining_PR_v2(existing_projects)

Number of missing PRS -->7
[u'4779d58229fbe5cb94e8c6c40204af0515c07677', u'156cd442edf6badb940149878cecfba2d3198776', u'cf35632382557982d526792bbc78d3e50888ff7e', u'12b641f181711ec1e9a50f1dcfc7861721aa3242', u'68c601d4a1fcc45f81f6bb5d2160ade962012ddf', u'26837ac3ab19dcd537ceb5805a4cbbfce205cc1b', u'ba50eafc653f8d6340bb261fd0da81dc1929a3be']
[u'azure-mgmt-servicebus', u'azure-mgmt-datalake-store', u'azure-mgmt-storage', u'azure.mgmt.datalake.analytics.account', u'azure-mgmt-documentdb', u'azure-mgmt-eventhub', u'azure-mgmt-consumption']
start ...0
end ...7
200
200
200
200
200
200
200
prs in the func
[[(u'azure-mgmt-servicebus', u'ab6034c2ed4ae7347a5817242487706e5a49b73c', u'1138'), (u'azure-mgmt-datalake-store', u'ab6034c2ed4ae7347a5817242487706e5a49b73c', u'1182'), (u'azure-mgmt-storage', u'ab6034c2ed4ae7347a5817242487706e5a49b73c', u'1120'), (u'azure.mgmt.datalake.analytics.account', u'ab6034c2ed4ae7347a5817242487706e5a49b73c', u'1214'), (u'azure-mgmt-documentdb', u'ab6034c2ed4ae7347a

[[(u'azure-mgmt-servicebus',
   u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
   u'1138'),
  (u'azure-mgmt-datalake-store',
   u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
   u'1182'),
  (u'azure-mgmt-storage',
   u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
   u'1120'),
  (u'azure.mgmt.datalake.analytics.account',
   u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
   u'1214'),
  (u'azure-mgmt-documentdb',
   u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
   u'1169'),
  (u'azure-mgmt-eventhub',
   u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
   u'1199'),
  (u'azure-mgmt-consumption',
   u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
   u'1192')]]

In [115]:
x = [u'4779d58229fbe5cb94e8c6c40204af0515c07677', u'156cd442edf6badb940149878cecfba2d3198776', u'cf35632382557982d526792bbc78d3e50888ff7e', u'12b641f181711ec1e9a50f1dcfc7861721aa3242', u'68c601d4a1fcc45f81f6bb5d2160ade962012ddf', u'26837ac3ab19dcd537ceb5805a4cbbfce205cc1b', u'ba50eafc653f8d6340bb261fd0da81dc1929a3be']
y = [u'azure-mgmt-servicebus', u'azure-mgmt-datalake-store', u'azure-mgmt-storage', u'azure.mgmt.datalake.analytics.account', u'azure-mgmt-documentdb', u'azure-mgmt-eventhub', u'azure-mgmt-consumption']

In [123]:
update_remaining_PR_v2(existing_projects)

Number of missing PRS -->7
[u'4779d58229fbe5cb94e8c6c40204af0515c07677', u'156cd442edf6badb940149878cecfba2d3198776', u'cf35632382557982d526792bbc78d3e50888ff7e', u'12b641f181711ec1e9a50f1dcfc7861721aa3242', u'68c601d4a1fcc45f81f6bb5d2160ade962012ddf', u'26837ac3ab19dcd537ceb5805a4cbbfce205cc1b', u'ba50eafc653f8d6340bb261fd0da81dc1929a3be']
[u'azure-mgmt-servicebus', u'azure-mgmt-datalake-store', u'azure-mgmt-storage', u'azure.mgmt.datalake.analytics.account', u'azure-mgmt-documentdb', u'azure-mgmt-eventhub', u'azure-mgmt-consumption']
start ...0
end ...7
200
200
200
200
200
200
200
prs in the func
[[(u'azure-mgmt-servicebus', u'4779d58229fbe5cb94e8c6c40204af0515c07677', u'1138'), (u'azure-mgmt-datalake-store', u'156cd442edf6badb940149878cecfba2d3198776', u'1182'), (u'azure-mgmt-storage', u'cf35632382557982d526792bbc78d3e50888ff7e', u'1120'), (u'azure.mgmt.datalake.analytics.account', u'12b641f181711ec1e9a50f1dcfc7861721aa3242', u'1214'), (u'azure-mgmt-documentdb', u'68c601d4a1fcc45f81

[[(u'azure-mgmt-servicebus',
   u'4779d58229fbe5cb94e8c6c40204af0515c07677',
   u'1138'),
  (u'azure-mgmt-datalake-store',
   u'156cd442edf6badb940149878cecfba2d3198776',
   u'1182'),
  (u'azure-mgmt-storage',
   u'cf35632382557982d526792bbc78d3e50888ff7e',
   u'1120'),
  (u'azure.mgmt.datalake.analytics.account',
   u'12b641f181711ec1e9a50f1dcfc7861721aa3242',
   u'1214'),
  (u'azure-mgmt-documentdb',
   u'68c601d4a1fcc45f81f6bb5d2160ade962012ddf',
   u'1169'),
  (u'azure-mgmt-eventhub',
   u'26837ac3ab19dcd537ceb5805a4cbbfce205cc1b',
   u'1199'),
  (u'azure-mgmt-consumption',
   u'ba50eafc653f8d6340bb261fd0da81dc1929a3be',
   u'1192')]]

In [118]:
get_prs_in_range(x[0:7], y[0:7])

200
azure-mgmt-servicebus ab6034c2ed4ae7347a5817242487706e5a49b73c
200
azure-mgmt-datalake-store ab6034c2ed4ae7347a5817242487706e5a49b73c
200
azure-mgmt-storage ab6034c2ed4ae7347a5817242487706e5a49b73c
200
azure.mgmt.datalake.analytics.account ab6034c2ed4ae7347a5817242487706e5a49b73c
200
azure-mgmt-documentdb ab6034c2ed4ae7347a5817242487706e5a49b73c
200
azure-mgmt-eventhub ab6034c2ed4ae7347a5817242487706e5a49b73c
200
azure-mgmt-consumption ab6034c2ed4ae7347a5817242487706e5a49b73c


[(u'azure-mgmt-servicebus',
  u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
  u'1138'),
 (u'azure-mgmt-datalake-store',
  u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
  u'1182'),
 (u'azure-mgmt-storage', u'ab6034c2ed4ae7347a5817242487706e5a49b73c', u'1120'),
 (u'azure.mgmt.datalake.analytics.account',
  u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
  u'1214'),
 (u'azure-mgmt-documentdb',
  u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
  u'1169'),
 (u'azure-mgmt-eventhub',
  u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
  u'1199'),
 (u'azure-mgmt-consumption',
  u'ab6034c2ed4ae7347a5817242487706e5a49b73c',
  u'1192')]

## HELPER FUNCTIONS

In [126]:
def get_prs_in_range(shas, projects):

    """
    Small helper function to return PR numbers given a 1) list of shas, and 2) corresponding project names. 
    returns a tuple of project, sha, and pr_number -->(u'azure-mgmt-servicebus', u'ab6034c2ed4ae7347a5817242487706e5a49b73c', u'1138')
    """

    return_list =[]    
    
    for i in range(len(projects)):
        r_com = shas[i]
        proj = projects[i]
        pr_num = get_pr_from_commits(r_com)
        
        if pr_num:
            if '#' in pr_num:
                pr_num = pr_num[1:]

            return_list.append((proj, r_com, pr_num))
            
        
        """"
        #bad idea to add no -pr as this may be because of a network error or 429 from github. 

        else:
            return_list.append((proj, r_com, 'No PR num found'))
            
        """

    return return_list

def update_remaining_PR_v2(existing_projects, max_lookup =50, sha2pr=None):
    """
    given a max number of PR nums to find (usually 9, as github limits 10 per min.) updates the missing PR #
    saves the commit_sha --> PR num relations in a json file (sha2pr_history.json) in order to minimize look up. 
    waits for 90 seconds for every 9 look up. 
    max_lookup value is safeguard to stop after 50 look ups (about 600 seconds)
    sha2pr is a dict of {'sha' : 'pr_num' } that can be updated and kept to minimize future look ups. 
    
    """


    if not sha2pr:
        sha2pr = {}
    
    for p in existing_projects:
        if existing_projects[p].get('changes'):
            if not existing_projects[p]['changes'].get('pr_num'):
                existing_projects[p]['changes']['pr_num'] = 'n.a'
            else:
                pr = existing_projects[p]['changes']['pr_num']
                if '#' in pr:
                    #remove the '#' from PR num. 
                    pr = pr[1:]
                
    
    missing_pr_shas =[]
    missing_pr_projects =[]
    

    for p in existing_projects:
        if existing_projects[p].get('changes'):
            if existing_projects[p]['changes'].get('commit_sha'):   
                if not existing_projects[p]['changes'].get('pr_num') or existing_projects[p]['changes']['pr_num'] == 'not found':
                    r_commit = existing_projects[p]['changes']['commit_sha'][0]
                    #check if r_commit is already is in the file.        
                    if sha2pr.get(r_commit):
                        existing_projects[proj]['changes']['pr_num'] = sha2pr[r_commit]
                        
                    else:
                        missing_pr_shas.append(r_commit)
                        missing_pr_projects.append(p)
                        
    print "Number of missing PRS -->" + str(len(missing_pr_shas))
    
    print (missing_pr_shas)
    print (missing_pr_projects)
    
    prs =[]
    remaining =0
    max_time = 0
    userange = len(missing_pr_projects)
    
    if len(missing_pr_projects) > 9:
        userange =9
        remaining = len(missing_pr_projects) - 9
        max_time +=9
        
        
    #update the first 9 projects. 
    start = 0
    end = userange
    
    print('start ...' + str(start))
    print('end ...' + str(end))

    prs.append(get_prs_in_range(missing_pr_shas[start:end], missing_pr_projects[start:end]))
    
    print('prs in the func')
    print(prs)

    if not remaining ==0:
        time.sleep(90) 

        while remaining > 0 and max_time < max_lookup:

            if remaining > 9:
                start = start + 9 
                end = end + 9 
                remaining = remaining - 9 
                max_time +=0
            else:
                start = start +9
                end = start + remaining 
                remaining = 0  

            prs.append(get_prs_in_range(missing_pr_shas[start:end], missing_pr_projects[start:end]))

            time.sleep(90) 

    return prs
    

def parse_swagger_to_sdk_config(project):
#count the #  of slashes 1 ->, composite file. , 3 =>swagger file with datefolder. > 3 staggered/subprojects. 
#Use the fact that folder -=2015, 2016, 2017. ..starts with 20

    sdk = project['output_dir'].split('/')[0]

    namespace = project['autorest_options']['Namespace']

    if not namespace:
        namespace = ''

    swagger_file_path = project['swagger']

    if not swagger_file_path:
        return None 

    if 'swagger' in swagger_file_path:
        #not a composite
        split_path = swagger_file_path.split('/swagger/')   
        azure_api = '/'.join(split_path[0].split('/')[0:-1])
        folder, swagger_name = split_path[0].split('/')[-1], split_path[-1]


    else:
        #is a composite file. 
        folder = 'Composite'
        split_path = swagger_file_path.split('/')
        azure_api, swagger_name = split_path[0], split_path[-1]

    #print azure_api, folder, swagger_name


    #print azure_api_spec_folder, date_folder, swagger_file
    return (azure_api, folder, swagger_name, sdk, namespace)

def get_pr_from_swagger_path(azure_api_swagger_path):

    swagger_soup_url = 'https://github.com/Azure/azure-rest-api-specs/blob/master/' + azure_api_swagger_path
    page = requests.get(swagger_soup_url)
    if page.status_code == 200:
        pagesoup = BeautifulSoup(page.content)
        
    #get_new_project_details(new_proj[0])
    page_pull = pagesoup.find("div", class_ ="commit-tease")
    if page_pull:
        issue = page_pull.find("a", class_="issue-link")
        if issue:
            return issue['href'].split('/')[-1]
        else:
            return 'not found'
    return 'not found'

#request_helper(raw_url + 'compositeComputeClient_2016_04_30_preview.json')
def update_multiple_projects_4_web(existing_changes):
    for p in existing_changes:
        if existing_changes[p].get('multiple') and existing_changes[p].get('parent_sdk'):
            parent = existing_changes[p]['parent_sdk']
            child = p
            if existing_changes.get(parent) and existing_changes[child].get('changes'):
                
                if not existing_changes[parent]['meta'].get('multiple_changes'):
                    existing_changes[parent]['meta']['multiple_changes'] = 'Yes'
                    
                if not existing_changes[parent].get('changes'):
                    existing_changes[parent]['changes'] = {'multiple_changes' : 'Yes'}
                 
                if not existing_changes[parent].get('multiple_projects'):
                    existing_changes[parent]['multiple_projects'] = {}

                if not existing_changes[parent]['multiple_projects'].get(child):
                    existing_changes[parent]['multiple_projects'][child] = {} 
                    existing_changes[parent]['multiple_projects'][child]['changes'] = existing_changes[child]['changes']
                else:
                     existing_changes[parent]['multiple_projects'][child]['changes'] = existing_changes[child]['changes']
                        
        if existing_changes[p].get('changes'):
            #composite project has ind_project with changes.
            #max swagger-behind to be reported correctly. 
            if existing_changes[p]['changes'].get('ind_changes'):
                max = existing_changes[p]['changes']['swagger_behind']
                for k,v in existing_changes[p]['changes']['ind_changes'].items():
                    if v['swagger_behind'] > max:
                        max = v['swagger_behind']
                existing_changes[p]['changes']['swagger_behind'] = max 

                        
def request_helper(url, access_token=None):
    """
    helper function/method to call API using request and return JSON encoded object. 
    if fails or gets 404, raises error. 
    
    """
    
    if not access_token:
        access_token = '2dd078a2a012e23bed1ff39015ead3675bc9f1d0'
        
    r = requests.get(url, auth=('username', access_token))
    
    if r.status_code != 200:
        return 
    
    else:
        return r.json()
    

def should_ignore(changed_file):
    """
    helper function to ignore certain files on azure_api_spec
    example: xyz.md, xyz.js, /test/ , /examples/ ....
    
    """
    if not changed_file.endswith('.json'):
        return True
    
    if changed_file.startswith('test/'):
        return True
    
    if '/examples/' in changed_file:
        return True
    
    if '/' not in changed_file:
        return True
    
    
    return False

#should_ignore('test/somethih')


def get_key_folder_params(git_url, azure_folder_path):
    """
    returns the composite status. if there is a .json file in root folder. this will be yes. 
    returns the most recent folder (2015-10-11 is more recent than 2014-10-09)
    """   
    rcomposite = request_helper(git_url + 'contents/' + azure_folder_path )
    most_recent_composite_status = 'No' 
    folders =[]
    for r in rcomposite:
        path = r['path']
        folder = path.split(azure_folder_path +'/')[1]
        if folder.startswith('20') or folder.startswith('/20'): 
            folders.append(folder)
        if '.json' in path:
            most_recent_composite_status = 'Yes'
            
    
    return(most_recent_composite_status, sorted(folders))

def get_key_folder_params_v2(git_url, azure_folder_path):
    """
    Given an azure api spec folder (which is a package) name, returns some key parameters for that package. 
    1) composite status. if there is a .json file in root folder. this will be yes. 
    2) most recent sub folder (2015-10-11 is more recent than 2014-10-09)
    3) gets the swagger file. ??
    """   
    rcomposite = request_helper(git_url + 'contents/' + azure_folder_path )
    most_recent_composite_status = 'No' 
    swagger = None
    folders =[]
    for r in rcomposite:
        path = r['path']
        folder = path.split(azure_folder_path +'/')[1]
        if folder.startswith('20') or folder.startswith('/20'): 
            folders.append(folder)
        
        print 'folder is ==='
        
        print folders
        
        if '.json' in path:
            most_recent_composite_status = 'Yes'
            swagger=path
            
    #print 'swagger is ===' + swagger
  
    if not swagger and folders:
        r_file = request_helper(git_url + 'contents/' + azure_folder_path + '/' + folders[-1] + '/swagger/')
        
        """
        print 'r_file is =='
        print r_file
        print 'folder is at ==='
        print git_url + 'contents/' + azure_folder_path + '/' + folders[-1] + '/Swagger/'
        """
        if not r_file:
            #try capital Swagger
            r_file2 = request_helper(git_url + 'contents/' + azure_folder_path + '/' + folders[-1] + '/Swagger/')
            if not r_file2:
                return ('No', ['not-found'], '')
            else:
                swagger =''
                for r in r_file2:
                    if r.get('path'):
                        #u'arm-timeseriesinsights/2017-02-28-preview/Swagger/timeseriesinsights.json'
                        swagger = r['path']
                        
        if r_file:
            swagger = ''
            for r in r_file:
                if '.json' in r:
                    swagger = r 
                    break;
                    
    return(most_recent_composite_status, sorted(folders), swagger)

def get_recent_from_nuget(package, base_url=None):
    """
    given a name of a C# package on Nuget ('Microsoft.Azure.Management.DataLake.Store'), 
    returns the most recent version and published date
    """
    #print package
    if not base_url:
        base_url = "https://www.nuget.org/packages/"
    
    page_result = requests.get(base_url + package)
    
    if page_result.status_code == 200:
        soup = BeautifulSoup(page_result.content)
        recent = soup.find("tr", class_= "versionTableRow recommended")
        recent_date, recent_release = '',''
        
        if recent:

                    recent_date = recent.find("span")['title']
                    recent_release = recent.find("a", href=True)['href'].split('/')[-1]
                    
        else:
            #get another set of tables. 
            recent = soup.findAll("tr", class_= "versionTableRow")
            #print recent
            if recent:
                recent_v = recent[0]
                recent_date = recent_v.find("span")['title']
                recent_release = recent_v.find("a", href=True)['href'].split('/')[-1]
    
    return (recent_date, recent_release)



def get_oldest_date_v2(file_dates, recent=None):
    
    """
    default -takes a list of dates, or a date, and returns the 'oldest' days and # of days old (i,e, 40 days ago)from today
    if recent=True, will return the 'most' recent date. 
    
    """
    
    date_format = "%Y-%m-%d"
    
    if type(file_dates) is list:
        sorted_dates = sorted(file_dates)
        if not recent:
            oldest_change = sorted_dates[0] #default sorting in ascending order. 
        else:
            oldest_change = sorted_dates[-1]
        
    else:
        oldest_change = file_dates
    
    a = datetime.strptime(datetime.now().strftime("%Y-%m-%d"), date_format)
    b = datetime.strptime(oldest_change.split('T')[0], date_format)

    #print str(abs(a-b))
    days = str(abs(a-b)).split(',')[0]

    return (oldest_change, days)

#get_oldest_date(file_dates)

def get_recent_from_nuget_v2(package, base_url=None):
    """
    given a name of a C# package on Nuget ('Microsoft.Azure.Management.DataLake.Store'), 
    returns the most recent version and published date
    """
    #print package
    if not base_url:
        base_url = "https://www.nuget.org/packages/"
    
    try:
        page_result = requests.get(base_url + package)
    except:
        return None
    
    d = { 'nuget_recent' : {}, 'nuget_stable' : {} }
    
    if page_result.status_code == 200:
        soup = BeautifulSoup(page_result.content)
        stable = soup.find("tr", class_= "versionTableRow recommended")
        recents = soup.findAll("tr", class_= "versionTableRow")
        recent_date, recent_release, stable_date, stable_release = '','', '', ''
        recent_href, stable_href = '',''
        
        if stable:
            stable_date = stable.find("span")['title']
            stable_days_ago = get_oldest_date_v2(stable_date)[1]
            stable_href = stable.find("a", href=True)['href']
            stable_release = stable_href.split('/')[-1]
            
            d['nuget_stable'] = { 'stable_date': stable_date, 'stable_release' : stable_release, 
                           'stable_href': stable_href, 'stable_days_ago' : stable_days_ago } 
            
                    
        if recents:
            recent = recents[0]
            recent_date = recent.find("span")['title']
            recent_days_ago = get_oldest_date_v2(recent_date)[1]
            recent_href = recent.find("a", href=True)['href']
            recent_release = recent_href.split('/')[-1]
            
            d['nuget_recent'] = { 'recent_date': recent_date, 'recent_release' : recent_release, 
               'recent_href': recent_href, 'recent_days_ago' : recent_days_ago }

            
    
    return d

def get_key_folder_params_v3(git_url, azure_folder_path):
    """
    Given an azure api spec folder (which is a package) name, returns some key parameters for that package. 
    1) composite status. if there is a .json file in root folder. this will be yes. 
    2) most recent sub folder (2015-10-11 is more recent than 2014-10-09)
    3) gets the swagger file. ??
    """   
    rcomposite = request_helper(git_url + 'contents/' + azure_folder_path )
    
    if not rcomposite:
        return None
    
    most_recent_composite_status = 'No' 
    swagger = None
    folders =[]
    for r in rcomposite:
        path = r['path']
        folder = path.split(azure_folder_path +'/')[1]
        if folder.startswith('20') or folder.startswith('/20'): 
            folders.append(folder)
        if '.json' in path:
            most_recent_composite_status = 'Yes'
            swagger=path
  
    if not swagger and folders:
        r_file = request_helper(git_url + 'contents/' + azure_folder_path + '/' + folders[-1] + '/swagger/')
        swagger =''
        for r in r_file:
            #print r
            if '.json' in r.get('name'):
                swagger = r.get('path')
        #swagger = r_file[0]['path']
        
            
    return(most_recent_composite_status, sorted(folders), swagger)

###GET build.json info from azure sdk page. #########

def get_python_sdk_build_info(sdk_name):
    
    """
    get the latest build.json from azure-sdk-python. 
    returns None if no build.json is found. 
    """
    
    sdk_url = 'https://api.github.com/repos/Azure/azure-sdk-for-python/'
    sdk_raw_url = 'https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/' 

    
    key_sdk_data = get_key_folder_params_v3(sdk_url,  sdk_name)

    if key_sdk_data and key_sdk_data[2] != None:
        #print key_sdk_data
        if 'build.json' in key_sdk_data[2]:
        #get file info
            build_file = request_helper(sdk_raw_url + key_sdk_data[2])

            if build_file and build_file['date']:
                build_info = { 'autorest' : build_file['autorest'], 'date' : build_file['date'], 
                              'version': build_file['version']}


            #print (build_info)

            return build_info 
    
    else:
        return 
    
def compare_build_date(build_date, dates_list):
        for i,d in enumerate(dates_list):
            if d < build_date:
                print (i, d)
                return (i, d)
                break;

def get_azure_name_space_data(swagger_file_path):
    #count the #  of slashes 1 ->, composite file. , 3 =>swagger file with datefolder. > 3 staggered/subprojects. 
    #Use the fact that folder -=2015, 2016, 2017. ..starts with 20
    
    split_path = swagger_file_path.split('/20')
    
    if len(split_path) > 1:
        #not composite. 
        azure_api_spec_folder, date_folder, swagger_file  = split_path[0], '20'+ split_path[1].split('/')[0], split_path[1].split('/')[2]
    
    else:
        azure_api_spec_folder, date_folder, swagger_file = split_path[0].split('/')[0], 'Composite',  split_path[0].split('/')[1]
        
    #print azure_api_spec_folder, date_folder, swagger_file
    return (azure_api_spec_folder, date_folder, swagger_file)

    
def get_new_project_names_v2(azure_projects_in_sdk, git_url=None, ignore_list=None):
    """
    given an existing list of project names (azure_projects_in_sdk,) in azure api spec namespaces
    returns a list of projects that are not on azure api spec github but not in the input list. 
    
    """
    if not ignore_list:
        ignore_list = ['.github', '.gitignore', '.travis.yml', '.vscode', 'LICENSE' , 'README.md' , 'azure-rest-api-specs.sln', 
          'azure-rest-api-specs.njsproj', 'documentation', 'package.json', '.scripts' , 'scripts']

    if not git_url:
        git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'


    api_folders = request_helper(git_url+'contents/')
    #print(api_folders)
    new_projects = []
    for folder in api_folders:
        f = folder['path'] #

        if f not in ignore_list: 
            test =0 
            for m in azure_projects_in_sdk:     
                if f in m:
                    test =1
                    break; 

            if test ==0:
                new_projects.append(f)
                #print ('New project added:' + f)

    #print(new_projects)
    return new_projects

def get_pr_from_commits(commit_sha, base_url=None, access_token=None):
    """
    given a sha of commit on azure api spec, returns a PR# on github  (if there is one)
    else, returns ' '
    """
    #print package
    #https://github.com/search?q=8461020530ea97978+repo%3AAzure%2Fazure-rest-api-specs&type=issues
    
    if not base_url:
        base_url = "https://github.com/search?q="
        url_en = "+repo%3AAzure%2Fazure-rest-api-specs&type=issues"
    
    url = base_url+commit_sha+url_en
    #print url
    
    if not access_token:
        access_token = '2dd078a2a012e23bed1ff39015ead3675bc9f1d0'
        
    
    page_result = requests.get(url, auth=('username', access_token))
    
    pr = ''
    
    print(page_result.status_code)
    
    if page_result.status_code == 200:
        soup = BeautifulSoup(page_result.content)
        try:
            issue = soup.find("div", class_= "issue-list")
            pr = issue.find("span", class_ ="float-right").contents[0]
        except:
            pass 
        
    
    return pr



## Core functions

In [42]:
#swagger history 
def get_swagger_updates_v2(azure_api_swagger_path, git_url=None, current_date=None):
    """
    return the updates to a swagger. 
    if a current_sha of a swagger is known, function returns the updates 'since' this current date
    else, function returns the entire swagger history. 

    #current_sha = base_data['current_version']

    """
    #object to return 
    
    changes= {}
    changes['file_dates'] = []
    changes['commit_sha'] = []
    changes['swagger_behind'] = 0
   
    #azure_api_swagger_path = 'keyvault/2016-10-01/swagger/keyvault.json'

    if not current_date:
        current_date = '2017-04-01'

    swagger_history = request_helper(git_url+'commits?path=' + azure_api_swagger_path + "&since=" + current_date)

    if not swagger_history:
        #print(changes)
        return changes

    #else there are changes. 

    #print(swagger_history)

    shas, dates  = [], [] 
    for s in swagger_history:
        shas.append(s['sha'])
        dates.append(s['commit']['committer']['date'])

    if shas:
        file_dates, commit_sha, swagger_behind = dates, shas, len(shas)

    else:
        #print(swagger_history)
        raise ValueError('Error: get_swagger_updates_v2 : There are updates since ' + current_date + 'but shas were not extracted')

    if file_dates:
        oldest, days_behind = get_oldest_date_v2(file_dates)
        changes['oldest_commit'] = oldest.split('T')[0] + ',' + days_behind +' ago'

    changes['file_dates'] = file_dates
    changes['commit_sha'] = commit_sha
    changes['swagger_behind'] = swagger_behind
    
    #get the pull path. 
    ##get PR from swagger. 
    
    pr_num = get_pr_from_swagger_path(azure_api_swagger_path)
    
    changes['pr_num'] = pr_num

    #print(changes)

    return changes

def get_swagger_updates(azure_api_swagger_path, git_url=None, current_sha=None):
    """
    return the updates to a swagger. 
    if a current_sha of a swagger is known, function returns the updates 'since' this current sha
    else, function returns the entire swagger history. 
    
    #current_sha = base_data['current_version']
    
    """
    swagger_history = request_helper(git_url+'commits?path=' + azure_api_swagger_path)
    
    shas, dates , prs = [], [] , []
    for s in swagger_history:
        shas.append(s['sha'])
        dates.append(s['commit']['committer']['date'])
        
        
    #compare the sha. 
    if current_sha:
        try:
            sha_index = shas.index(current_sha)
            if sha_index > 0: 
                #i.e. there are more recent shas. 
                print('new commits discoverd for -> ' + 'index =' + str(sha_index))
                print('corresponding_date' + str(dates[sha_index]))

                file_dates, commit_sha, swagger_behind = dates[:sha_index], shas[:sha_index] , sha_index
                

            else:
                file_dates, commit_sha, swagger_behind =[],[],0
                
        except ValueError, Argument:
            print "sha index not found", Argument
            file_dates, commit_sha, swagger_behind = dates, shas, len(shas)
            
    else:
        file_dates, commit_sha, swagger_behind = dates, shas, len(shas)
    
    
    
    #create an object to return 
    changes= {}
    
    if file_dates:
        oldest, days_behind = get_oldest_date_v2(file_dates)
        changes['oldest_commit'] = oldest.split('T')[0] + ',' + days_behind +' ago'
        #print file_dates
        #changes['pulls'] = [get_pr_from_commits(c) for c in commit_sha]
        
    changes['file_dates'] = file_dates
    changes['commit_sha'] = commit_sha
    changes['swagger_behind'] = swagger_behind
    
    return changes

def get_new_project_names(map_object, git_url=None, ignore_list=None):
    
    if not ignore_list:
        ignore_list = ['.github', '.gitignore', '.travis.yml', '.vscode', 'LICENSE' , 'README.md' , 'azure-rest-api-specs.sln', 
          'azure-rest-api-specs.njsproj', 'documentation', 'package.json', '.scripts' , 'scripts']
        
    if not git_url:
        git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'
        
        
    api_folders = request_helper(git_url+'contents/')
    
    new_projects = []
    for folder in api_folders:
        f = folder['path'] #

        if f not in ignore_list: 
            test =0 
            for m in map_object:     
                if f in m:
                    test =1
                    break; 

            if test ==0:
                new_projects.append(f)
                #print ('New project added:' + f)
    return new_projects

def get_new_project_details(new_projects_list, git_url=None):
    
    if not git_url:
        git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'
        
    
    new_output ={}
    for p in new_projects_list:
        #print (get_key_folder_params(git_url,p))
        is_composite, folders, swagger = get_key_folder_params_v2(git_url,p)
        if not new_output.get(p):

            new_output[p] ={}
            new_output[p]['is_composite'] = is_composite
            new_output[p]['latest_folder'] = folders[-1]
            new_output[p]['swagger'] = swagger
            new_output[p]['commits'] = []
            new_output[p]['commit_dates'] = []

            #get swagger file history
            r_files = request_helper(git_url+'commits?path=' + swagger)
            for r in r_files:
                commit_sha = r['sha']
                commit_date = r['commit']['committer']['date']
                new_output[p]['commits'].append(commit_sha)
                new_output[p]['commit_dates'].append(commit_date)
                
            #get the oldest commit 
            
            new_output[p]['oldest_commit'] =''
            if new_output[p]['commit_dates']:
                oldest, days_behind = get_oldest_date_v2(new_output[p]['commit_dates'])
                new_output[p]['oldest_commit']= oldest.split('T')[0] + "," + days_behind + " ago "
        
                 

    #print(new_output)
    return(new_output)

#print(get_new_project_details(get_new_project_names()))

#def get_existing_changes(sdk_map, current_project_details, git_url=None):
##EXISTING CHANGES ###

def get_existing_changes_v3(sdk_map, swagger_to_sdk_config_file_name =None, sdk_url=None, git_url=None, sdk_raw_url=None, raw_url=None, assumed_current_date=None):
    
    """
    sdk_url = https://api.github.com/repos/Azure/azure-sdk-for-python/
    
    """
    if not raw_url:
        raw_url = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/master/' 
    if not sdk_raw_url:
        sdk_raw_url = 'https://raw.githubusercontent.com/Azure/azure-sdk-for-python/master/' 


    if not swagger_to_sdk_config_file_name:
        swagger_to_sdk_config_file_name = 'swagger_to_sdk_config.json'

    if not assumed_current_date:
        assumed_current_date = '2017-04-01' #all packages without build.json are assued to be current as of  04-01

    if not git_url:
        git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'


    if not sdk_url:
        sdk_url = 'https://api.github.com/repos/Azure/azure-sdk-for-python/'


    existing_changes ={}
    existing_changes['errors'] = {}

    #get main swagger_to_sdk_config file 

    swagger_to_sdk = request_helper(sdk_raw_url + swagger_to_sdk_config_file_name )


    for proj in swagger_to_sdk['projects']:

        #if proj.startswith('rdbms.mysql'):

        print ('printing_prject_name :' + proj )

        #(u'arm-commerce', u'2015-06-01-preview', u'commerce.json'), (u'arm-network', 'Composite', u'compositeNetworkClient_2015_06_15.json')

        azure_api_name, c_composite, c_swagger, sdk, namespace = parse_swagger_to_sdk_config(swagger_to_sdk['projects'][proj])

        current_swagger_path = swagger_to_sdk['projects'][proj]['swagger']

        sdk_recent_build = get_python_sdk_build_info(sdk)

        #print(sdk_recent_build)

        if not sdk_recent_build:
            c_recent_date = assumed_current_date #assume it's current as of April 15th. 
            c_version = swagger_to_sdk['projects'][proj]['autorest_options'].get("PackageVersion", "0.00") #assume current version == package version

        else:
            c_recent_date = sdk_recent_build['date']
            c_version = sdk_recent_build['version']

        #print c_recent_date, c_version

        if not existing_changes.get(sdk):
            i = sdk 
            existing_changes[i] ={}
            existing_changes[i]['meta'] = {'azure_api_name' : azure_api_name, 'composite_or_recent_folder' : c_composite, 
                               'current_swagger': current_swagger_path , 'recent_build_date': c_recent_date, 
                               'current_version':c_version}

        else:
            #there is already an object with the same sdk name. ..this is multiple. 
            i= namespace
            existing_changes[i] = {}
            existing_changes[i]['meta'] = {'azure_api_name' : azure_api_name, 'composite_or_recent_folder' : c_composite, 
                   'current_swagger': c_swagger , 'recent_build_date': c_recent_date, 
                   'current_version':c_version}
            existing_changes[i]['multiple'] = 'yes'
            existing_changes[i]['parent_sdk'] = sdk

        #GET THE info from azure api spec for this project. 

        params = get_key_folder_params_v3(git_url,azure_api_name)

        if not params:
            existing_changes[i]['errors'] = {'Project' : 'Project ' + azure_api_name +' not found. Probably private' }
            existing_changes['errors'][i] = 'Project ' + azure_api_name +' not found. Probably private'

        else:
            is_composite, folders, swagger = get_key_folder_params_v3(git_url,azure_api_name)

            if c_composite =='Composite':
                current_composite = 'Yes'
                c_recent_folder = 'No'

            else:
                c_recent_folder = c_composite
                current_composite = 'No'

            #print(azure_api_name, c_composite, c_swagger)
            #print(is_composite, folders, swagger )


            #check changes in composite status. 
            if is_composite != current_composite:
                #project composite status has changed. 
                changes = get_swagger_updates(swagger, git_url=git_url) 
                changes['use_swagger'] = swagger

                if current_composite =="Yes":
                    #is_composite is NO

                    print(azure_api_name + 'has changed to Composite to --> non composite')  

                    #get the details for this swagger file and update the details. 

                    changes['change_type'] = "CompositeStatus"
                    changes['change_status'] = "Moved to a non - composite swagger"
                    existing_changes[i]['changes'] = changes


                else:
                    #moved to a composite swagger. get all lower level swagger details if possible. 

                    print(azure_api_name + 'Moved to a composite swagger')
                    changes['change_type'] = "CompositeStatus"
                    changes['change_status'] = "Moved to a composite swagger"

                    existing_changes[i]['changes']= changes

            #check if folder is the same 
            else:     
                if is_composite =='No': 
                    #there must be folders. 
                    if len(folders) > 0: 
                        latest_folder = folders[-1]
                        if c_recent_folder != latest_folder:

                            print(azure_api_name + ' has a new folder : ' + latest_folder)

                            changes = get_swagger_updates(swagger, git_url=git_url)
                            changes['use_swagger'] = swagger
                            changes['change_type'] = "Folder"
                            changes['new_folder'] = latest_folder
                            existing_changes[i]['changes'] =changes

                        else:
                            #most proable scenario : check for swagger update. 

                            if current_swagger_path != swagger:
                                print (azure_api_name + '   swagger not found')
                                #print(c_swagger, swagger)

                            else:
                                #get the current sha of from the recent date. 

                                changes = get_swagger_updates_v2(current_swagger_path, git_url=git_url, current_date=c_recent_date)

                                if changes['swagger_behind'] >0:
                                    changes['change_type'] = "SwaggerUpdate"
                                    existing_changes[i]['changes'] = changes


                else:
                    #TO DO TO DO TO DO existing composite project that may have been updated. check if swagger has changed. 
                    #i.e, a composite project. 
                    #current_sha = base_data['current_version']
                    print ('COMPOSITE_SWAGGER' + current_swagger_path)
                    if not c_swagger or not current_swagger_path:
                            print (azure_api_name + 'swagger not found')
                    else:
                        # check for the update in main composite file first 
                        changes_composite = get_swagger_updates_v2(current_swagger_path, git_url=git_url, current_date=c_recent_date)

                        if changes_composite['swagger_behind'] >0:
                            changes['change_type'] = "SwaggerUpdate"
                            existing_changes[i]['changes'] = changes

                        # check for the update in main composite file first                 
                        #get the full file and individual paths from the composite file. 
                        #raw_url = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/master/' 

                        cfile= request_helper(raw_url + current_swagger_path)

                        if not cfile:
                            #current swagger file not found. 

                            existing_changes['errors'][i] = 'swagger_file : ' + current_swagger_path +' not found'

                            if not existing_changes[i].get('errors'): 
                                existing_changes[i]['errors'] = {}
                                existing_changes[i]['errors']['swagger_file'] = 'swagger_file : ' + current_swagger_path +' not found'

                            else:
                                existing_changes[i]['errors']['swagger_file'] = 'swagger_file : ' + current_swagger_path +' not found'


                        if cfile and cfile.get('documents'):
                            for f in cfile.get('documents'):
                                #get file history 
                                ind_swagger = azure_api_name + f[1:]
                                print ('IND_SWAGGER' + ind_swagger)
                                #file_data = get_swagger_updates(ind_swagger, git_url, current_sha=None)
                                changes_ind_file = get_swagger_updates_v2(ind_swagger, git_url=git_url, current_date=c_recent_date)

                                #catch any errors. 

                                if not changes_ind_file:
                                    existing_changes['errors'][i] = 'swagger_file : ' + ind_swagger +' not found'

                                if changes_ind_file and changes_ind_file['swagger_behind'] >0:

                                    if existing_changes[i].get('changes'):

                                        if not existing_changes[i]['changes'].get('ind_changes'):
                                            existing_changes[i]['changes']['ind_changes'] = {}
                                            existing_changes[i]['changes']['ind_changes'][ind_swagger] = changes_ind_file
                                        else:
                                             existing_changes[i]['changes']['ind_changes'][ind_swagger] = changes_ind_file


                                #d[ind_swagger] = {'sha' : file_data['commit_sha'], 'dates': file_data['file_dates']}

            #print('CHANGES')

            if not sdk_map.get(azure_api_name):
                existing_changes[i]['nuget_info'] = {}
                print('No Nuget URL Map for azure api: ' + azure_api_name)

            else:
                nuget_package = sdk_map[azure_api_name].get('nuget_package')
                if nuget_package:
                    existing_changes[i]['nuget_info'] = get_recent_from_nuget_v2(nuget_package) or 'Nuget info not found.'
                else:
                    existing_changes[i]['nuget_info'] = {}
                    print('No Nuget URL Map for azure api: ' + azure_api_name)

        #update multiple projects for easy rendering later on. 

        update_multiple_projects_4_web(existing_changes)

    return existing_changes


## SCRATCH WORK BELOW

In [ ]:
print(get_python_sdk_build_info('azure-mgmt-cdn'))
with open('config/api2sdk2nuget.json', 'r') as f:
    sdk_map = json.load(f)

In [ ]:
sorted_sdk_projects = sorted(sdk_projects)
real_sdk = lambda x:'.'.join(x.split('.')[:-1])
for i,s in enumerate(sorted_sdk_projects):
    if '.20' in s:
        real_s = real_sdk(s)
        print s, real_sdk
        if real_s == real_sdk(sorted_sdk_projects[i+1]):
            #remove i 
            del sorted_sdk_projects[i]

sorted_sdk_projects       
        

In [ ]:
test_sdk = 'azure-graphrbac'
get_python_sdk_build_info(test_sdk)


In [ ]:
import os, json,shutil
import requests, jinja2
import random
from bs4 import BeautifulSoup
from time import gmtime, strftime
from datetime import datetime

"""
base_url = "https://www.nuget.org/packages/"
package = 'Microsoft.Azure.Management.DataLake.Store'
page_result = requests.get(base_url + package)
"""
#globals (via a config later on)
git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'

raw_url = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/master/' 
#cfile= request_helper(raw_url + x[-1])

#load base files (map and latest build file to compare against. )

with open('config/api2sdk2nuget.json', 'r') as f:
    map_object = json.load(f)

with open('config/fudge.json', 'r') as f:
    build_file = json.load(f)

In [ ]:
comp_projs = [c for c in build_file['projects'] if build_file['projects'][c].get('swagger_meta')['is_composite'] == 'Yes' ]

In [ ]:
comp_projs = [u'graphrbac',
 u'web',
 u'network',
 u'monitor.data',
 u'sql',
 u'compute',
 u'monitor.mgmt']

In [ ]:
#update the build so we have the necessary data. 
for cp in comp_projs:
    #get the info 
    
    """
        'swagger_path_a' : {
        "sha": [ ],
        "dates" :[]
    },

        'swagger_path_b' : {
        "sha": [ ],
        "dates" :[]
    },

}
    """
    #print cp
    
    d ={}
    comp_file = build_file['projects'][cp]['swagger']
    azure_api_name = build_file['projects'][cp]['swagger_meta']['azure_api_spec_folder']
    #print comp_file
    raw_url = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/master/' 
    cfile= request_helper(raw_url + comp_file)
    
    print cfile
    
    if cfile.get('documents'):
        for f in cfile.get('documents'):
            #get file history 
            ind_swagger = azure_api_name + f[1:]
            print ind_swagger
            file_data = get_swagger_updates(ind_swagger, git_url, current_sha=None)
            d[ind_swagger] = {'sha' : file_data['commit_sha'], 'dates': file_data['file_dates']}
            
            #print d
            
            build_file['projects'][cp]['individual_file_history'] = d 
            

with open('config/build002.json', 'w') as f:
    json.dump(build_file, f)
    

In [ ]:
soup = BeautifulSoup(page_result.content)

In [ ]:
stable = soup.find("tr", class_= "versionTableRow recommended")

In [ ]:
stable

In [ ]:
recent = soup.findAll("tr", class_= "versionTableRow")

In [ ]:
recent[0].find("span")['title']

In [ ]:
recent[0].find("a", href=True)['href']

In [ ]:
package = 'Microsoft.Azure.Management.DataLake.Store'
get_recent_from_nuget_v2(package)

In [ ]:
#globals (via a config later on)
git_url = 'https://api.github.com/repos/Azure/azure-rest-api-specs/'
test_comp_folder ='arm-network'

In [ ]:
get_key_folder_params_v3(git_url, test_comp_folder)

In [ ]:
x = ('Yes',
 [u'2015-05-01-preview',
  u'2015-06-15',
  u'2016-03-30',
  u'2016-06-01',
  u'2016-09-01',
  u'2016-12-01',
  u'2017-03-01'],
 u'arm-network/compositeNetworkClient.json')

In [ ]:
sorted_folders = x[1]


In [ ]:
#get the swagger file names. 
#for each file, get the history ..l
#compare the history as usual. current_date of individual swagger == current_date of the composite file. 
raw_url = 'https://raw.githubusercontent.com/Azure/azure-rest-api-specs/master/' 
cfile= request_helper(raw_url + x[-1])

    

In [ ]:
cfile['documents']

In [ ]:
test_swagger = 'arm-network' + '/2017-03-01/swagger/applicationGateway.json'
get_swagger_updates(test_swagger, git_url, current_sha=None)

In [ ]:
with open('changes/latest.json') as f:
    test = json.load(f)


In [ ]:
for t in test['existing_projects']:
    #print t
    if test['existing_projects'][t].get('changes'):
        print t

In [ ]:
for key in sorted(existing_projects.iterkeys()):
    print key